# Fit models for illustration #1

In [ ]:
import numpy as np
from scipy.io import loadmat
import glob, os

from scipy.io import savemat # store results for comparison with Matlab code   

from __future__ import division
import matplotlib.pyplot as plt

from pybasicbayes.distributions import Regression
from pybasicbayes.util.text import progprint_xrange
from autoregressive.distributions import AutoRegression

import pyximport
pyximport.install()

absolute_code_path = '/home/mackelab/Desktop/Projects/Stitching/code/pyLDS_dev/'
os.chdir(absolute_code_path +'pylds')

from models import LDS, DefaultLDS
from distributions import Regression_diag, AutoRegression_input
from obs_scheme import ObservationScheme
from user_util import gen_pars, rand_rotation_matrix, init_LDS_model, collect_LDS_stats

from scipy.io import savemat # store results for comparison with Matlab code   
from scipy.linalg import solve_discrete_lyapunov as dtlyap # solve discrete-time Lyapunov equation

relative_data_path = '../../../results/cosyne_poster/illustration_1/data'
os.chdir(relative_data_path)
filenames = glob.glob("*.npz")
num_exps = len(filenames)
idx_exps = range(num_exps)

eps = np.log(1.01)
max_iter = 1000

initialiser = 'params'

for i in idx_exps:
    
    filename = filenames[i]

    os.chdir('../data/')    
    loadfile = np.load(filename)

    data = loadfile['y']
    T,p = data.shape
    n   = loadfile['x'].shape[1]
    obs_scheme = ObservationScheme(p=p, T=T, 
                                   sub_pops=tuple([item for item in loadfile['sub_pops']]),
                                   obs_pops=loadfile['obs_pops'], 
                                   obs_time=loadfile['obs_time'])
    pars_true = loadfile['truePars'].reshape(1,)[0] # some numpy (cross-version 3.x-2.x?)     
    tmp = {}
    for j in range(len(pars_true.keys())):
        tmp[str(pars_true.keys()[j])] =  pars_true.values()[j]
    pars_true = tmp    

    print('(T, p, n, eps) = ', (T, p, eps))
    
    os.chdir('../init/')
    
    loadfile = np.load('init_'+filename)
    
    pars_init = loadfile[initialiser].reshape(1,)[0]
    tmp = {}
    for j in range(len(pars_init.keys())):
        tmp[str(pars_init.keys()[j])] =  pars_init.values()[j]
    pars_init = tmp    

    pars_init['R'] = np.diag(pars_init['R'])

    
    def update(model):
        model.EM_step()
        return model.log_likelihood()
    
    ###################
    #    EM cycles    #
    ###################


    # get EM-step results after m iterations                    
    model = init_LDS_model(pars_init, data, obs_scheme) # reset to initialisation                    
    print 'fitting #' + str(i)    
    likes = [-np.inf]
    for t in progprint_xrange(max_iter):
        likes.append(update(model))
        if likes[-1]-likes[-2] < eps:
            break
        
    stats_hat,pars_hat = collect_LDS_stats(model)

    # get EM-step results from true parameters
    model = init_LDS_model(pars_true, data, obs_scheme) # reset to true pars
    model.E_step()
    stats_true,_ = collect_LDS_stats(model)
    model.M_step()                                        

    ###################
    #  store results  #
    ###################

    save_file_m = {'x': model.states_list[0].stateseq, 
                   'y': model.states_list[0].data,
                   'u' : [], 
                   'll' : likes, 
                   'T' : model.states_list[0].T, 
                   'Trial': len(model.states_list), 
                   'ifUseB':False, 
                   'ifUseA':True, 
                   'epsilon':eps,
                   'truePars':pars_true,
                   'initPars':pars_init,
                   'estPars': pars_hat,
                   'stats_h': stats_hat,
                   'stats_true': stats_true,
                   'Pi':dtlyap(pars_true['A'], pars_true['Q']),
                   'Pi_h':dtlyap(pars_hat['A'], pars_hat['Q']),
                   'Pi_t':pars_true['A'].dot(dtlyap(pars_true['A'], pars_true['Q'])),
                   'Pi_t_h':pars_hat['A'].dot(dtlyap(pars_hat['A'], pars_hat['Q'])),
                   'obsScheme' : obs_scheme}
    
    print('finished in ' + str(len(likes)-1) + ' many steps.')

    os.chdir('../fits/')
    savemat(initialiser + '_' + filename,save_file_m) # does the actual saving


In [ ]:
# get EM-step results after m iterations                    
model = init_LDS_model(pars_init, data, obs_scheme) # reset to initialisation                    
print 'fitting #' + str(i)    
likes = [-np.inf]
for t in progprint_xrange(max_iter):
    likes.append(update(model))
    if likes[-1]-likes[-2] < eps:
        break

# Fit models for simulation #1

In [ ]:
import numpy as np
from scipy.io import loadmat
import glob, os

from scipy.io import savemat # store results for comparison with Matlab code   

from __future__ import division
import matplotlib.pyplot as plt

from pybasicbayes.distributions import Regression
from pybasicbayes.util.text import progprint_xrange
from autoregressive.distributions import AutoRegression

import pyximport
pyximport.install()

absolute_code_path = '/home/mackelab/Desktop/Projects/Stitching/code/pyLDS_dev/'
os.chdir(absolute_code_path +'pylds')

from models import LDS, DefaultLDS
from distributions import Regression_diag, AutoRegression_input
from obs_scheme import ObservationScheme
from user_util import gen_pars, rand_rotation_matrix, init_LDS_model, collect_LDS_stats

from scipy.io import savemat # store results for comparison with Matlab code   
from scipy.linalg import solve_discrete_lyapunov as dtlyap # solve discrete-time Lyapunov equation

relative_data_path = '../../../results/cosyne_poster/simulation/data'
os.chdir(relative_data_path)
filenames = glob.glob("*.npz")
num_exps = len(filenames)
idx_exps = range(num_exps)

eps = np.log(1.01)
max_iter = 1000

initialiser = 'params_naive'

for i in idx_exps:
    
    filename = filenames[i]

    os.chdir('../data/')    
    loadfile = np.load(filename)

    data = loadfile['y']
    T,p = data.shape
    n   = loadfile['x'].shape[1]
    obs_scheme = ObservationScheme(p=p, T=T, 
                                   sub_pops=tuple([item for item in loadfile['sub_pops']]),
                                   obs_pops=loadfile['obs_pops'], 
                                   obs_time=loadfile['obs_time'])
    pars_true = loadfile['truePars'].reshape(1,)[0] # some numpy (cross-version 3.x-2.x?)     
    tmp = {}
    for i in range(len(pars_true.keys())):
        tmp[str(pars_true.keys()[i])] =  pars_true.values()[i]
    pars_true = tmp    

    print('(T, p, n, eps) = ', (T, p, eps))
    
    os.chdir('../init/')
    
    loadfile = np.load('init_'+filename)
    
    pars_init = loadfile[initialiser].reshape(1,)[0]
    tmp = {}
    for i in range(len(pars_init.keys())):
        tmp[str(pars_init.keys()[i])] =  pars_init.values()[i]
    pars_init = tmp    

    pars_init['R'] = np.diag(pars_init['R'])

    
    def update(model):
        model.EM_step()
        return model.log_likelihood()
    
    ###################
    #    EM cycles    #
    ###################


    # get EM-step results after m iterations                    
    model = init_LDS_model(pars_init, data, obs_scheme) # reset to initialisation                    
    print 'fitting #' + str(i)    
    likes = [-np.inf]
    for t in progprint_xrange(max_iter):
        likes.append(update(model))
        if likes[-1]-likes[-2] < eps:
            break
        
    stats_hat,pars_hat = collect_LDS_stats(model)

    # get EM-step results from true parameters
    model = init_LDS_model(pars_true, data, obs_scheme) # reset to true pars
    model.E_step()
    stats_true,_ = collect_LDS_stats(model)
    model.M_step()                                        

    ###################
    #  store results  #
    ###################

    save_file_m = {'x': model.states_list[0].stateseq, 
                   'y': model.states_list[0].data,
                   'u' : [], 
                   'll' : likes, 
                   'T' : model.states_list[0].T, 
                   'Trial': len(model.states_list), 
                   'ifUseB':False, 
                   'ifUseA':True, 
                   'epsilon':eps,
                   'truePars':pars_true,
                   'initPars':pars_init,
                   'estPars': pars_hat,
                   'stats_h': stats_hat,
                   'stats_true': stats_true,
                   'Pi':dtlyap(pars_true['A'], pars_true['Q']),
                   'Pi_h':dtlyap(pars_hat['A'], pars_hat['Q']),
                   'Pi_t':pars_true['A'].dot(dtlyap(pars_true['A'], pars_true['Q'])),
                   'Pi_t_h':pars_hat['A'].dot(dtlyap(pars_hat['A'], pars_hat['Q'])),
                   'obsScheme' : obs_scheme}
    
    print('finished in ' + str(len(likes)-1) + ' many steps.')

    os.chdir('../fits/')
    savemat(initialiser + '_' + filename,save_file_m) # does the actual saving
